## Mayank Raj
### 22BAI1118

In [7]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, load_model  
image_directory = '/kaggle/input/all-images/images_by_emotion'
csv_file = '/kaggle/input/emotionss/images_emotions.csv'

data = pd.read_csv(csv_file)

image_names = data['Image_Name'].values  
emotions = data['Emotion'].values  

emotion_dict = {emotion: idx for idx, emotion in enumerate(np.unique(emotions))}
print("Emotion Dictionary:", emotion_dict)

labels = np.array([emotion_dict[emotion] for emotion in emotions])

images = []
for img_name in image_names:
    img_path = os.path.join(image_directory, img_name)  
    img = cv2.imread(img_path)  
    if img is not None:  
        img = cv2.resize(img, (48, 48))  
        images.append(img)
    else:
        print(f"Warning: Image {img_name} not found.")

X = np.array(images)
Y = np.array(labels)

X = X.astype('float32') / 255.0

Y = to_categorical(Y, num_classes=len(emotion_dict))

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}")


Emotion Dictionary: {'ANGRY': 0, 'CONTEMPT': 1, 'DISGUST': 2, 'FEAR': 3, 'JOY': 4, 'NEUTRAL': 5, 'SAD': 6, 'SURPRISE': 7}
Training data shape: (1792, 48, 48, 3), Validation data shape: (448, 48, 48, 3)


## Custom CNN

In [21]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(emotion_dict), activation='softmax'))  
    return model

model = create_model()
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 46, 46, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 356,552 (1.36 MB)

 Trainable params: 356,552 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [2]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('best_cnn_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')

history = model.fit(X_train, Y_train, epochs=200, validation_data=(X_val, Y_val), batch_size=32, callbacks=[model_checkpoint])

best_model = load_model('best_cnn_model.keras')
val_loss, val_accuracy = best_model.evaluate(X_val, Y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


Emotion Dictionary: {'ANGRY': 0, 'CONTEMPT': 1, 'DISGUST': 2, 'FEAR': 3, 'JOY': 4, 'NEUTRAL': 5, 'SAD': 6, 'SURPRISE': 7}
Training data shape: (1792, 48, 48, 3), Validation data shape: (448, 48, 48, 3)
Epoch 1/200


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.2094 - loss: 1.9185 - val_accuracy: 0.2411 - val_loss: 1.8508
Epoch 2/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2201 - loss: 1.8601 - val_accuracy: 0.2835 - val_loss: 1.7966
Epoch 3/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2409 - loss: 1.8130 - val_accuracy: 0.2857 - val_loss: 1.7885
Epoch 4/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2619 - loss: 1.7874 - val_accuracy: 0.2902 - val_loss: 1.7810
Epoch 5/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2543 - loss: 1.8034 - val_accuracy: 0.2857 - val_loss: 1.7633
Epoch 6/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3038 - loss: 1.7543 - val_accuracy: 0.3125 - val_loss: 1.7674
Epoch 7/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2838 - loss: 1.7793 - val_accuracy: 0.2768 - val_loss: 1.7523
Epoch 8/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3086 - loss: 1.7267 - val_accuracy: 0.3080 - val_loss: 1.

## Simple MobileNetV2

In [22]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x) 
predictions = Dense(len(emotion_dict), activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('best_mobilenet_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')

model.summary()

Model: "functional_67"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_12[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,268,232 (8.65 MB)

 Trainable params: 10,248 (40.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [3]:

history = model.fit(X_train, Y_train, batch_size=32,
                    epochs=200,
                    validation_data=(X_val, Y_val),
                    callbacks=[model_checkpoint])
best_model_V2 = load_model('best_mobilenet_model.keras')

val_loss, val_accuracy = best_model_V2.evaluate(X_val, Y_val)
print(f"Best Model Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")



Emotion Dictionary: {'ANGRY': 0, 'CONTEMPT': 1, 'DISGUST': 2, 'FEAR': 3, 'JOY': 4, 'NEUTRAL': 5, 'SAD': 6, 'SURPRISE': 7}
Training data shape: (1792, 224, 224, 3), Validation data shape: (448, 224, 224, 3)
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 16s 111ms/step - accuracy: 0.1957 - loss: 2.1352 - val_accuracy: 0.2723 - val_loss: 1.8879
Epoch 2/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.3362 - loss: 1.7342 - val_accuracy: 0.2902 - val_loss: 1.8070
Epoch 3/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.3951 - loss: 1.5828 - val_accuracy: 0.2790 - val_loss: 1.8497
Epoch 4/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.4254 - loss: 1.4845 - val_accuracy: 0.2857 - val_loss: 1.8064
Epoch 5/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.5077 - loss: 1.3632 - val_accuracy: 0.2879 - val_loss: 1.8225
Epoch 6/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.5077 - loss: 1.3300 - val_accuracy: 0

## Complex MobileNetV2

In [23]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = BatchNormalization()(x)  

x = Dense(512)(x)
x = BatchNormalization()(x)  
x = Activation('relu')(x) 
x = Dropout(0.5)(x)  

x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(len(emotion_dict), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('best_mobilenet_complex_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
model.summary()

Model: "functional_69"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_13[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,055,432 (11.66 MB)

 Trainable params: 793,352 (3.03 MB)

 Non-trainable params: 2,262,080 (8.63 MB)

In [6]:

history = model.fit(X_train, Y_train, batch_size=32,
                    epochs=200,
                    validation_data=(X_val, Y_val),
                    callbacks=[model_checkpoint])

best_model_CV2 = load_model('best_mobilenet_complex_model.keras')

val_loss, val_accuracy = best_model_CV2.evaluate(X_val, Y_val)
print(f"Best Model Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")



Emotion Dictionary: {'ANGRY': 0, 'CONTEMPT': 1, 'DISGUST': 2, 'FEAR': 3, 'JOY': 4, 'NEUTRAL': 5, 'SAD': 6, 'SURPRISE': 7}
Training data shape: (1792, 224, 224, 3), Validation data shape: (448, 224, 224, 3)
Epoch 1/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 16s 105ms/step - accuracy: 0.1935 - loss: 2.5504 - val_accuracy: 0.3304 - val_loss: 1.8092
Epoch 2/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.3755 - loss: 1.7437 - val_accuracy: 0.3259 - val_loss: 1.7772
Epoch 3/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.4238 - loss: 1.5876 - val_accuracy: 0.3013 - val_loss: 1.7814
Epoch 4/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.4830 - loss: 1.4226 - val_accuracy: 0.3438 - val_loss: 1.7728
Epoch 5/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.5475 - loss: 1.2325 - val_accuracy: 0.3192 - val_loss: 1.8683
Epoch 6/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.5950 - loss: 1.1120 - val_accuracy: 0.3482 - val_loss: 1.8762
Epoch 7/200
56/56 ━━━━━━

## Simple VGG16

In [24]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  
predictions = Dense(len(emotion_dict), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('best_vgg16_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
model.summary()

Model: "functional_71"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_10     │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 8)              │         4,104 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,718,792 (56.15 MB)

 Trainable params: 4,104 (16.03 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [5]:

history = model.fit(X_train, Y_train, batch_size=32,
                    epochs=200,
                    validation_data=(X_val, Y_val),
                    callbacks=[model_checkpoint])

best_model_VGG16 = load_model('best_vgg16_model.keras')

val_loss, val_accuracy = best_model_VGG16.evaluate(X_val, Y_val)
print(f"Best Model Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

Emotion Dictionary: {'ANGRY': 0, 'CONTEMPT': 1, 'DISGUST': 2, 'FEAR': 3, 'JOY': 4, 'NEUTRAL': 5, 'SAD': 6, 'SURPRISE': 7}
Training data shape: (1792, 224, 224, 3), Validation data shape: (448, 224, 224, 3)
Epoch 1/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 190ms/step - accuracy: 0.1942 - loss: 2.2671 - val_accuracy: 0.2478 - val_loss: 1.8578
Epoch 2/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 174ms/step - accuracy: 0.2054 - loss: 1.9718 - val_accuracy: 0.2812 - val_loss: 1.8346
Epoch 3/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 180ms/step - accuracy: 0.2400 - loss: 1.9085 - val_accuracy: 0.2902 - val_loss: 1.8099
Epoch 4/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 179ms/step - accuracy: 0.2593 - loss: 1.8230 - val_accuracy: 0.3036 - val_loss: 1.7952
Epoch 5/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 174ms/step - accuracy: 0.2448 - loss: 1.8529 - val_accuracy: 0.3192 - val_loss: 1.7855
Epoch 6/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 166ms/step - accuracy: 0.2660 - loss: 1.7981 - val_accuracy: 0.2969 - val_loss: 1.7770
Epoch 7/200
56/

## Complex VGG16

In [25]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x) 
x = BatchNormalization()(x) 
x = Dropout(0.5)(x) 
predictions = Dense(len(emotion_dict), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers[:15]:  
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('_complex_vgg16_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
model.summary()

Model: "functional_73"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_11     │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 512)            │             

 Total params: 14,983,496 (57.16 MB)

 Trainable params: 7,347,208 (28.03 MB)

 Non-trainable params: 7,636,288 (29.13 MB)

In [1]:

history = model.fit(X_train, Y_train, batch_size=32,
                    epochs=200,
                    validation_data=(X_val, Y_val),
                    callbacks=[model_checkpoint])

best_model_complex_vgg16 = load_model('_complex_vgg16_model.keras')

val_loss, val_accuracy = best_model_complex_vgg16.evaluate(X_val, Y_val)
print(f"Best Model Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


Emotion Dictionary: {'ANGRY': 0, 'CONTEMPT': 1, 'DISGUST': 2, 'FEAR': 3, 'JOY': 4, 'NEUTRAL': 5, 'SAD': 6, 'SURPRISE': 7}
Training data shape: (1792, 224, 224, 3), Validation data shape: (448, 224, 224, 3)
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/200


I0000 00:00:1728480115.414600     102 service.cc:145] XLA service 0x7813c4113a90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728480115.414683     102 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1728480115.414689     102 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 1/56 ━━━━━━━━━━━━━━━━━━━━ 17:35 19s/step - accuracy: 0.0938 - loss: 3.7116

I0000 00:00:1728480132.252609     102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


56/56 ━━━━━━━━━━━━━━━━━━━━ 30s 205ms/step - accuracy: 0.1514 - loss: 3.2175 - val_accuracy: 0.1853 - val_loss: 3.5955
Epoch 2/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 187ms/step - accuracy: 0.1779 - loss: 2.7230 - val_accuracy: 0.2210 - val_loss: 2.7512
Epoch 3/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 175ms/step - accuracy: 0.1933 - loss: 2.5169 - val_accuracy: 0.1830 - val_loss: 2.2760
Epoch 4/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 188ms/step - accuracy: 0.2008 - loss: 2.3825 - val_accuracy: 0.2455 - val_loss: 1.8841
Epoch 5/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 182ms/step - accuracy: 0.2296 - loss: 2.2577 - val_accuracy: 0.2455 - val_loss: 1.8169
Epoch 6/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 195ms/step - accuracy: 0.2416 - loss: 2.1822 - val_accuracy: 0.2746 - val_loss: 1.9552
Epoch 7/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 189ms/step - accuracy: 0.2310 - loss: 2.1749 - val_accuracy: 0.2522 - val_loss: 1.8612
Epoch 8/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 11s 194ms/step - accuracy: 0.2288 - loss: 2.0879 - val_accuracy

### Creating desired CSV using 4th model: complex VGG16

In [18]:
import os
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Model

image_directory = '/kaggle/input/face-emotion-recognition-1/TestSet/TestSet'
image_names = [img for img in os.listdir(image_directory) if img.endswith(('.jpg', '.jpeg', '.png'))]

csv_file_path = '/kaggle/working/image_predictions.csv'
df = pd.DataFrame(image_names, columns=['Image_Name'])
df.to_csv(csv_file_path, index=False)

print(f"CSV file created at: {csv_file_path}")

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(8, activation='softmax')(x)  
model = Model(inputs=base_model.input, outputs=predictions)

model.load_weights('/kaggle/working/_complex_vgg16_model.keras')

emotion_dict = {'ANGRY': 0, 'CONTEMPT': 1, 'DISGUST': 2, 'FEAR': 3, 'JOY': 4, 'NEUTRAL': 5, 'SAD': 6, 'SURPRISE': 7}

predicted_emotions = []
confidence_levels = []
x_coords = []
y_coords = []
widths = []
heights = []

haarcascade_path = '/kaggle/input/model-1/keras/default/1/haarcascade_frontalface_default.xml'

face_cascade = cv2.CascadeClassifier(haarcascade_path)
for img_name in image_names:
    img_path = os.path.join(image_directory, img_name)
    
    img = cv2.imread(img_path)
    if img is not None:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        img_resized = cv2.resize(img, (224, 224))
        img_array = img_to_array(img_resized) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        
        predicted_emotion = list(emotion_dict.keys())[list(emotion_dict.values()).index(predicted_class)]

        predicted_emotions.append(predicted_emotion)
        confidence_levels.append(confidence)

        if len(faces) > 0:  
            for (x, y, w, h) in faces:
                x_coords.append(x)
                y_coords.append(y)
                widths.append(w)
                heights.append(h)
                break  
        else:
            x_coords.append(0) 
            y_coords.append(0)
            widths.append(0)
            heights.append(0)

df['Predicted_Emotion'] = predicted_emotions
df['Confidence_Level'] = confidence_levels
df['X_Coordinate'] = x_coords
df['Y_Coordinate'] = y_coords
df['Width'] = widths
df['Height'] = heights

df.to_csv(csv_file_path, index=False)
print(f"Results saved to: {csv_file_path}")


CSV file created at: /kaggle/working/image_predictions.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19m

# Conclucion
### In my study of facial emotion recognition, I evaluated several models to determine which architecture yielded the best performance. The results showed that the Complex VGG16 model achieved the highest validation accuracy of 39.96%, outperforming other models. While it had a higher validation loss compared to some, its superior accuracy indicates it is better at classifying emotions in the dataset. The Complex VGG16 model's ability to capture intricate patterns in facial images makes it the most effective choice for this task. Therefore, I chose the Complex VGG16 model for its enhanced performance and potential for further improvement in facial emotion recognition.